In [105]:
import numpy as np
import math
from copy import deepcopy

K = 2
alpha = 1
beta = 1

def load_data_set(filename):
    # your code
    tokens = []
    tags = []
    f = open(filename, 'r')
    for line in f:
        pairs = line.split()
        current_tokens = []
        current_tags = []
        for pair in pairs:
            data = pair.split('/')
            current_tokens.append(data[0])
            current_tags.append(data[1])
        tokens.append(current_tokens)
        tags.append(current_tags)
    return tokens, tags

def preprocessing(tokens,tags):
    k = K
    dic = {}
    prepocessed_tags = deepcopy(tags)
    for sentence in tokens:
        for token in sentence:
            if token in dic:
                dic[token] += 1
            else:
                dic[token] = 1
    for i in range(len(tokens)):
        for j in range(len(tokens[i])):
            if dic[tokens[i][j]] < k:
                prepocessed_tags[i][j] = "UNK"
    return prepocessed_tags,dic

def get_probilities(tokens,tags,x_dic):
    y_dic = {}
    y_y_dic = {}
    for tag_set in tags:
        new_tag_set = list(tag_set)
        new_tag_set.insert(0, 'start')
        new_tag_set.append('end')
        
        for i in range(len(new_tag_set)):
            if new_tag_set[i] in y_dic:
                y_dic[new_tag_set[i]] += 1
            else:
                y_dic[new_tag_set[i]] = 1
            
            if i == len(new_tag_set) - 1:
                break
            
            if new_tag_set[i]+'|'+new_tag_set[i+1] in y_y_dic:
                y_y_dic[new_tag_set[i]+'|'+new_tag_set[i+1]] += 1
            else:
                y_y_dic[new_tag_set[i]+'|'+new_tag_set[i+1]] = 1
#     print(y_dic,y_y_dic)
    p_y_y = y_y_dic.copy()
    for y_y in y_y_dic.keys():
        p_y_y[y_y] = 0.0
        p_y_y[y_y] = (y_y_dic[y_y]+alpha)/(y_dic[y_y.split('|')[0]]+alpha*(len(y_dic)+1))
    
    x_y_dic = {}
    for i in range(len(tokens)):
        for j in range(len(tokens[i])):
            if tokens[i][j]+'|'+tags[i][j] in x_y_dic:
                x_y_dic[tokens[i][j]+'|'+tags[i][j]] += 1
            else:
                x_y_dic[tokens[i][j]+'|'+tags[i][j]] = 1
    
    p_x_y = {}
    for x_y in x_y_dic.keys():
        p_x_y[x_y] = 0.0
        p_x_y[x_y] = (x_y_dic[x_y]+beta)/(y_dic[x_y.split('|')[1]]+beta*len(x_dic))    
    
#     count = 0.0
#     for x_y in p_x_y.keys():
#         if x_y.split('|')[1] == 'C':
#             count += p_x_y[x_y]
#     print(count)
    
#     count = 0.0
#     for y_y in p_y_y.keys():
#         if y_y[0] == 'P':
#             count += p_y_y[y_y]
#     print(count)
    return p_y_y,p_x_y

def to_logistic(p_y_y,p_x_y):
    log_p_y_y = p_y_y.copy()
    log_p_x_y = p_x_y.copy()
    for key in log_p_y_y:
        log_p_y_y[key] = math.log2(log_p_y_y[key])
    for key in log_p_x_y:
        log_p_x_y[key] = math.log2(log_p_x_y[key])
    return log_p_y_y,log_p_x_y

def Viterbi(log_p_y_y,log_p_x_y):
    
    

trn_tokens, trn_tags = load_data_set("./data/trn.pos")
prepocessed_tags,x_dic = preprocessing(trn_tokens,trn_tags)

# print(len(trn_texts[0]),len(trn_tags[0]))
p_y_y,p_x_y = get_probilities(trn_tokens,prepocessed_tags,x_dic)
log_p_y_y,log_p_x_y = to_logistic(p_y_y,p_x_y)